In [8]:
import findspark
from setuptools.command.alias import alias

findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Credit Card").getOrCreate()
from pyspark.sql.functions import *


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 12:43:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
loan = spark.read.csv("loan.csv", header = True, inferSchema = True)

In [10]:
loan.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

In [11]:
loan.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [12]:
len(loan.columns)

15

In [13]:
loan.count()

500

In [14]:
loan.distinct().count()

500

In [15]:
#number of loans per categroy
loan.groupBy("Loan Category").count().orderBy("count", ascending=False).show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|         GOLD LOAN|   77|
|           HOUSING|   67|
|        AUTOMOBILE|   60|
|        TRAVELLING|   53|
|       RESTAURANTS|   41|
|COMPUTER SOFTWARES|   35|
|          SHOPPING|   35|
|          BUSINESS|   24|
|  EDUCATIONAL LOAN|   20|
|        RESTAURANT|   20|
|           DINNING|   14|
|       ELECTRONICS|   14|
|   HOME APPLIANCES|   14|
|       AGRICULTURE|   12|
|       BOOK STORES|    7|
|          BUILDING|    7|
+------------------+-----+



In [17]:
# number of people who have taken more than 100 000 dollars loan
loan.filter(loan["Loan Amount"]>"1,00,000").count()

379

In [18]:
loan.filter(loan["Income"]>60000).count()

198

In [27]:
loan.withColumnRenamed(" Returned Cheque", "Returned Cheque")

DataFrame[Customer_ID: string, Age: int, Gender: string, Occupation: string, Marital Status: string, Family Size: int, Income: int, Expenditure: int, Use Frequency: int, Loan Category: string, Loan Amount: string, Overdue: int,  Debt Record: string, Returned Cheque: int,  Dishonour of Bill: int]

In [29]:
loan.filter((loan[" Returned Cheque"]>"1")&(loan["Income"]<"50000")).count()

137

In [31]:
loan.filter((loan[" Returned Cheque"]>1)&(loan["Marital Status"]=="SINGLE")).count()

111

In [34]:
loan.filter(loan["Expenditure"]>50000).show()

+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|     Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14158| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|   AUTOMOBILE| 20,45,789 |      1|      16,599|               2|                 3|
|    IB14176| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 80000|      62541|            2|      HOUSING| 20,45,789 |      1|      16,599|               2|                 3|
|    IB14204| 54|  MALE|AIRPORT OFFICER|       MARRIED|          6| 81000|      

In [36]:
credit_card = spark.read.csv("credit card.csv", header = True , inferSchema = True)
credit_card.show()
credit_card.printSchema()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

In [37]:
len(credit_card.columns)

13

In [38]:
credit_card.count()

10000

In [39]:
credit_card.distinct().count()

10000

In [40]:
credit_card.filter(credit_card["CreditScore"]>700).count()

3116

In [41]:
credit_card.filter((credit_card["CreditScore"]>700)&(credit_card["IsActiveMember"]==1)).count()

1637

In [42]:
credit_card.filter(credit_card["Geography"]=="Spain").count()

2477

In [45]:
txn = spark.read.csv("txn.csv", header = True, inferSchema= True)
txn.show()
txn.printSchema()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7500000.0|

In [56]:
txn.groupBy("Account No").max(" WITHDRAWAL AMT ").orderBy("max( WITHDRAWAL AMT )", ascending = False).show()

+-------------+---------------------+
|   Account No|max( WITHDRAWAL AMT )|
+-------------+---------------------+
|     1196711'|        4.594475464E8|
|409000438620'|                4.0E8|
|409000425051'|               3.54E8|
|409000438611'|                2.4E8|
|409000405747'|                1.7E8|
|     1196428'|                1.5E8|
|409000362497'|        1.413662392E8|
|409000493210'|                1.5E7|
|409000493201'|            2500000.0|
|409000611074'|             912000.0|
+-------------+---------------------+



In [58]:
txn.groupBy("Account No").min(" WITHDRAWAL AMT ").orderBy("min( WITHDRAWAL AMT )").show()

+-------------+---------------------+
|   Account No|min( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000493210'|                 0.01|
|409000438611'|                  0.2|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000438620'|                 0.34|
|409000362497'|                 0.97|
|409000425051'|                 1.25|
|409000493201'|                  2.1|
|409000405747'|                 21.0|
|409000611074'|                120.0|
+-------------+---------------------+



In [60]:
txn.groupBy("Account No").max(" DEPOSIT AMT ").orderBy("max( DEPOSIT AMT )", ascending = False).show()

+-------------+------------------+
|   Account No|max( DEPOSIT AMT )|
+-------------+------------------+
|409000438620'|           5.448E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000405747'|           2.021E8|
|409000362497'|             2.0E8|
|409000438611'|          1.7025E8|
|409000493210'|             1.5E7|
|409000425051'|             1.5E7|
|409000611074'|         3000000.0|
|409000493201'|         1000000.0|
+-------------+------------------+



In [68]:
txn.groupBy("Account No").agg(sum("BALANCE AMT").alias("Balance")).orderBy("Balance", ascending = False).show()

+-------------+--------------------+
|   Account No|             Balance|
+-------------+--------------------+
|409000611074'|       1.615533622E9|
|409000493201'|1.0420831829499985E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000438611'|-2.49486577068339...|
|409000493210'|-3.27584952132095...|
|409000438620'|-7.12291867951358...|
|     1196711'|-1.60476498101275E13|
|409000362497'| -5.2860004792808E13|
|     1196428'| -8.1418498130721E13|
+-------------+--------------------+



In [75]:
txn.groupBy("VALUE DATE").agg(count("*").alias("rank")).orderBy("rank", ascending =  False).show()

+----------+----+
|VALUE DATE|rank|
+----------+----+
| 27-Jul-17| 567|
| 13-Aug-18| 463|
|  8-Nov-17| 402|
|  7-Oct-17| 382|
| 10-Jul-18| 374|
| 12-Dec-17| 367|
| 12-Sep-18| 365|
|  9-Aug-18| 360|
| 19-Sep-17| 358|
| 16-Mar-17| 353|
| 10-Sep-18| 344|
| 14-Jul-17| 333|
|  7-Mar-18| 319|
| 11-Oct-18| 303|
| 22-Aug-17| 301|
|  9-Jan-18| 299|
|  9-Oct-18| 297|
| 20-Apr-18| 296|
|  9-Jul-18| 292|
|  7-Apr-18| 291|
+----------+----+
only showing top 20 rows



In [76]:
txn.select("Account No", "TRANSACTION DETAILS", " WITHDRAWAL AMT ").filter(txn[" WITHDRAWAL AMT "]>100000).show()

+-------------+--------------------+----------------+
|   Account No| TRANSACTION DETAILS| WITHDRAWAL AMT |
+-------------+--------------------+----------------+
|409000611074'|INDO GIBL Indiafo...|        133900.0|
|409000611074'|INDO GIBL Indiafo...|        195800.0|
|409000611074'|INDO GIBL Indiafo...|        143800.0|
|409000611074'|INDO GIBL Indiafo...|        331650.0|
|409000611074'|INDO GIBL Indiafo...|        129000.0|
|409000611074'|INDO GIBL Indiafo...|        230013.0|
|409000611074'|INDO GIBL Indiafo...|        367900.0|
|409000611074'|INDO GIBL Indiafo...|        108000.0|
|409000611074'|INDO GIBL Indiafo...|        141000.0|
|409000611074'|INDO GIBL Indiafo...|        206000.0|
|409000611074'|INDO GIBL Indiafo...|        242300.0|
|409000611074'|INDO GIBL Indiafo...|        113250.0|
|409000611074'|INDO GIBL Indiafo...|        206900.0|
|409000611074'|INDO GIBL Indiafo...|        276000.0|
|409000611074'|INDO GIBL Indiafo...|        171000.0|
|409000611074'|INDO GIBL Ind

In [98]:
txn.createOrReplaceTempView("Transactions")

list = spark.sql("""
select 'Account No', 'TRANSACTION DETAILS', 'WITHDRAWAL AMT' 
from Transactions
where 'WITHDRAWAL AMT' == 1000 """)

list.show()

+----------+-------------------+--------------+
|Account No|TRANSACTION DETAILS|WITHDRAWAL AMT|
+----------+-------------------+--------------+
+----------+-------------------+--------------+



In [90]:
txn.groupBy("Account NO").count().show()

+-------------+-----+
|   Account NO|count|
+-------------+-----+
|409000438611'| 4588|
|     1196711'|10536|
|     1196428'|48779|
|409000493210'| 6014|
|409000611074'| 1093|
|409000425051'|  802|
|409000405747'|   51|
|409000493201'| 1044|
|409000438620'|13454|
|409000362497'|29840|
+-------------+-----+

